# Progetto Airoldi — Gruppo 7
## Confronto Heston vs Merton su Superficie di Volatilità Sintetica

| Parametro | Valore |
|-----------|--------|
| $S_0$ | 100 |
| $r$ | 5% |
| $q$ | 0 |
| $T$ | 0.5 e 1.0 anno |
| Strike | 70–140 |

In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize, brentq
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11

S0, r, q = 100.0, 0.05, 0.0

## Parte I — Superficie di volatilità sintetica

Definiamo lo smile come funzione della log-moneyness $m = \ln(K/S_0)$:

$$\sigma_{mkt}(K, T) = 0.20 - 0.12\,\tanh(4m) + 0.05\,e^{-15m^2} + \frac{0.01}{\sqrt{T}}$$

Questo produce uno **skew negativo** (vol più alta per K bassi) e un'**ala destra quasi piatta**.

In [ ]:
def sigma_mkt(K, T):
    m = np.log(K / S0)
    return 0.20 - 0.12*np.tanh(4*m) + 0.05*np.exp(-15*m**2) + 0.01/np.sqrt(T)

def bs_call(S, K, T, r, sig):
    d1 = (np.log(S/K) + (r + 0.5*sig**2)*T) / (sig*np.sqrt(T))
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d1 - sig*np.sqrt(T))

def bs_iv(price, K, T):
    try: return brentq(lambda s: bs_call(S0, K, T, r, s) - price, 0.01, 2.0)
    except: return np.nan

strikes = np.linspace(70, 140, 15)
T1, T2 = 1.0, 0.5

# Plot smile
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
for T, col in [(T2, 'blue'), (T1, 'red')]:
    vols = [sigma_mkt(K, T) for K in strikes]
    ax.plot(strikes, [v*100 for v in vols], 'o-', color=col, label=f'T={T}Y')
ax.set_xlabel('Strike K'); ax.set_ylabel('Vol Implicita (%)')
ax.set_title('Superficie di Volatilità Sintetica'); ax.legend(); ax.grid(alpha=0.3)
plt.show()

# Salva target
target_iv, target_price = {}, {}
for T in [T2, T1]:
    for K in strikes:
        sv = sigma_mkt(K, T)
        target_iv[(K,T)] = sv
        target_price[(K,T)] = bs_call(S0, K, T, r, sv)

## Parte II — Funzioni caratteristiche e pricing COS

In [ ]:
# --- COS method generico ---
def cos_call(cf_fn, K, T, a=-5, b=5, N=256):
    x = np.log(S0/K)
    k = np.arange(N, dtype=float)
    w = k*np.pi/(b-a)
    cf = cf_fn(w, T)
    # Chi e Psi per call: payoff su [0, b]
    chi = np.where(k==0, np.exp(b)-1.0,
        (np.cos(w*(b-a))*np.exp(b)-1+w*np.sin(w*(b-a))*np.exp(b))/(1+w**2))
    psi = np.where(k==0, b, (b-a)/(k*np.pi)*np.sin(w*(b-a)))
    Hk = (2/(b-a))*(chi - psi)
    s = np.real(cf * np.exp(1j*k*np.pi*(x-a)/(b-a))) * Hk
    s[0] *= 0.5
    return K*np.exp(-r*T)*np.sum(s)

# --- Heston CF (di log(S_T/S_0)) ---
def heston_cf(u, T, pars):
    v0, kappa, vbar, sigv, rho = pars
    xi = kappa - 1j*rho*sigv*u
    d = np.sqrt(xi**2 + sigv**2*(1j*u + u**2))
    g = (xi-d)/(xi+d+1e-30)
    C = kappa*vbar/sigv**2 * ((xi-d)*T - 2*np.log((1-g*np.exp(-d*T))/(1-g+1e-30)))
    D = (xi-d)/sigv**2 * (1-np.exp(-d*T))/(1-g*np.exp(-d*T)+1e-30)
    return np.exp(1j*u*(r-q)*T + C + D*v0)

# --- Merton CF (di log(S_T/S_0)) ---
def merton_cf(u, T, pars):
    sigd, lam, muj, sigj = pars
    omega = -0.5*sigd**2 - lam*(np.exp(muj + 0.5*sigj**2) - 1)
    return np.exp(1j*u*(r-q+omega)*T - 0.5*sigd**2*u**2*T
                  + lam*T*(np.exp(1j*u*muj - 0.5*sigj**2*u**2) - 1))

# Test
hp0 = [0.04, 2.0, 0.04, 0.5, -0.7]
mp0 = [0.15, 0.5, -0.1, 0.2]
print(f"Heston ATM call: {cos_call(lambda u,T: heston_cf(u,T,hp0), 100, 1.0):.4f}")
print(f"Merton ATM call: {cos_call(lambda u,T: merton_cf(u,T,mp0), 100, 1.0):.4f}")

## Parte IV — Calibrazione su due maturità

In [ ]:
def calib_obj(pars, cf_maker):
    err = 0
    for T in [T2, T1]:
        for K in strikes:
            try:
                p = cos_call(lambda u,Tt: cf_maker(u,Tt,pars), K, T)
                iv = bs_iv(p, K, T)
                if np.isnan(iv): return 1e6
                err += (iv - target_iv[(K,T)])**2
            except: return 1e6
    return err

print("Calibrando Heston (può richiedere ~30s)...")
res_h = minimize(lambda p: calib_obj(p, heston_cf), hp0,
    method='Nelder-Mead', options={'maxiter':15000, 'xatol':1e-8, 'fatol':1e-10})
h_par = res_h.x
print(f"  v0={h_par[0]:.4f}, k={h_par[1]:.4f}, vbar={h_par[2]:.4f}, "
      f"sigv={h_par[3]:.4f}, rho={h_par[4]:.4f}  (err={res_h.fun:.2e})")

print("Calibrando Merton...")
res_m = minimize(lambda p: calib_obj(p, merton_cf), mp0,
    method='Nelder-Mead', options={'maxiter':15000, 'xatol':1e-8, 'fatol':1e-10})
m_par = res_m.x
print(f"  sig={m_par[0]:.4f}, lam={m_par[1]:.4f}, muj={m_par[2]:.4f}, "
      f"sigj={m_par[3]:.4f}  (err={res_m.fun:.2e})")

In [ ]:
# Confronto smile calibrato vs mercato
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for idx, T in enumerate([T2, T1]):
    iv_m = [target_iv[(K,T)]*100 for K in strikes]
    iv_h = [bs_iv(cos_call(lambda u,Tt: heston_cf(u,Tt,h_par),K,T),K,T)*100 for K in strikes]
    iv_mt = [bs_iv(cos_call(lambda u,Tt: merton_cf(u,Tt,m_par),K,T),K,T)*100 for K in strikes]
    axes[idx].plot(strikes, iv_m, 'ko-', ms=5, label='Mercato')
    axes[idx].plot(strikes, iv_h, 'b^--', ms=5, label='Heston')
    axes[idx].plot(strikes, iv_mt, 'rs--', ms=5, label='Merton')
    axes[idx].set_xlabel('Strike K'); axes[idx].set_ylabel('Vol Impl (%)')
    axes[idx].set_title(f'Smile calibrato — T = {T}Y')
    axes[idx].legend(); axes[idx].grid(alpha=0.3)
plt.tight_layout(); plt.show()

## Parte V — Simulazione Monte Carlo

In [ ]:
N_paths = 100000
N_steps = 500
T_mc = T1
dt = T_mc / N_steps
np.random.seed(42)

# --- MC Heston (Euler) ---
v0_h, kap_h, vb_h, sv_h, rho_h = h_par
S_hes = S0 * np.ones(N_paths)
v_hes = v0_h * np.ones(N_paths)
S_hes_path = np.zeros((N_paths, N_steps+1))
S_hes_path[:,0] = S0

for i in range(N_steps):
    Z1 = np.random.randn(N_paths)
    Z2 = rho_h*Z1 + np.sqrt(1-rho_h**2)*np.random.randn(N_paths)
    v_pos = np.maximum(v_hes, 0)
    S_hes = S_hes * np.exp((r-q-0.5*v_pos)*dt + np.sqrt(v_pos*dt)*Z1)
    v_hes = v_hes + kap_h*(vb_h - v_pos)*dt + sv_h*np.sqrt(v_pos*dt)*Z2
    S_hes_path[:,i+1] = S_hes

# --- MC Merton (GBM + Poisson jumps) ---
sd_m, lam_m, muj_m, sj_m = m_par
omega_m = -0.5*sd_m**2 - lam_m*(np.exp(muj_m + 0.5*sj_m**2) - 1)
S_mer = S0 * np.ones(N_paths)
S_mer_path = np.zeros((N_paths, N_steps+1))
S_mer_path[:,0] = S0

for i in range(N_steps):
    Z = np.random.randn(N_paths)
    N_jumps = np.random.poisson(lam_m*dt, N_paths)
    J = np.sum([np.random.normal(muj_m, abs(sj_m), N_paths) for _ in range(max(N_jumps.max(),1))], axis=0)
    J = np.where(N_jumps > 0, np.array([np.sum(np.random.normal(muj_m, abs(sj_m), nj)) if nj>0 else 0 for nj in N_jumps]), 0)
    S_mer = S_mer * np.exp((r-q+omega_m-0.5*sd_m**2)*dt + sd_m*np.sqrt(dt)*Z + J)
    S_mer_path[:,i+1] = S_mer

print(f"Heston MC: E[S_T]={np.mean(S_hes):.2f} (teorico {S0*np.exp(r*T_mc):.2f})")
print(f"Merton MC: E[S_T]={np.mean(S_mer):.2f} (teorico {S0*np.exp(r*T_mc):.2f})")

In [ ]:
# Plot traiettorie
t_grid = np.linspace(0, T_mc, N_steps+1)
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
for j in range(min(200, N_paths)):
    axes[0].plot(t_grid, S_hes_path[j], alpha=0.05, color='blue')
    axes[1].plot(t_grid, S_mer_path[j], alpha=0.05, color='red')
axes[0].set_title('Heston — Traiettorie MC'); axes[0].set_xlabel('t'); axes[0].set_ylabel('S')
axes[1].set_title('Merton — Traiettorie MC'); axes[1].set_xlabel('t'); axes[1].set_ylabel('S')
for ax in axes: ax.grid(alpha=0.3); ax.axhline(S0, color='k', ls='--', alpha=0.3)
plt.tight_layout(); plt.show()

### Verifica coerenza: MC vs prezzi vanilla di calibrazione

In [ ]:
print(f"{'K':>6} | {'Mkt':>8} | {'Heston COS':>11} | {'Heston MC':>10} | {'Merton COS':>11} | {'Merton MC':>10}")
print('-'*70)
for K in [80, 90, 100, 110, 120]:
    c_mkt = target_price[(K, T1)]
    c_h_cos = cos_call(lambda u,T: heston_cf(u,T,h_par), K, T1)
    c_m_cos = cos_call(lambda u,T: merton_cf(u,T,m_par), K, T1)
    c_h_mc = np.exp(-r*T1) * np.mean(np.maximum(S_hes - K, 0))
    c_m_mc = np.exp(-r*T1) * np.mean(np.maximum(S_mer - K, 0))
    print(f"{K:6.0f} | {c_mkt:8.4f} | {c_h_cos:11.4f} | {c_h_mc:10.4f} | {c_m_cos:11.4f} | {c_m_mc:10.4f}")

## Parte VI — Opzioni esotiche

In [ ]:
# --- Opzione Asiatica (media aritmetica, T=1Y) ---
avg_hes = np.mean(S_hes_path, axis=1)  # media su tutto il path
avg_mer = np.mean(S_mer_path, axis=1)

K_asian = S0  # ATM
asian_hes = np.exp(-r*T1) * np.mean(np.maximum(avg_hes - K_asian, 0))
asian_mer = np.exp(-r*T1) * np.mean(np.maximum(avg_mer - K_asian, 0))

# --- Down-and-Out Call (barriera monitorata, B=80, K=100) ---
B = 80.0
K_do = S0
# Monitoraggio a 0.5Y e 1Y
i_05 = N_steps // 2
i_10 = N_steps

# Barriera non toccata se S(0.5) > B AND S(1.0) > B
alive_hes = (S_hes_path[:, i_05] > B) & (S_hes_path[:, i_10] > B)
alive_mer = (S_mer_path[:, i_05] > B) & (S_mer_path[:, i_10] > B)

do_hes = np.exp(-r*T1) * np.mean(np.maximum(S_hes - K_do, 0) * alive_hes)
do_mer = np.exp(-r*T1) * np.mean(np.maximum(S_mer - K_do, 0) * alive_mer)

# Vanilla call per confronto
van_hes = np.exp(-r*T1) * np.mean(np.maximum(S_hes - K_do, 0))
van_mer = np.exp(-r*T1) * np.mean(np.maximum(S_mer - K_do, 0))

print(f"{'Opzione':>25} | {'Heston MC':>10} | {'Merton MC':>10} | {'Differenza':>10}")
print('-'*65)
print(f"{'Vanilla Call (K=100)':>25} | {van_hes:10.4f} | {van_mer:10.4f} | {van_hes-van_mer:+10.4f}")
print(f"{'Asian Call (K=100)':>25} | {asian_hes:10.4f} | {asian_mer:10.4f} | {asian_hes-asian_mer:+10.4f}")
print(f"{'D&O Call (K=100,B=80)':>25} | {do_hes:10.4f} | {do_mer:10.4f} | {do_hes-do_mer:+10.4f}")

In [ ]:
# Grafico riassuntivo
labels = ['Vanilla Call', 'Asian Call', 'D&O Call']
hes_vals = [van_hes, asian_hes, do_hes]
mer_vals = [van_mer, asian_mer, do_mer]

x_pos = np.arange(len(labels))
fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(x_pos - 0.15, hes_vals, 0.3, label='Heston', color='steelblue')
ax.bar(x_pos + 0.15, mer_vals, 0.3, label='Merton', color='coral')
ax.set_xticks(x_pos); ax.set_xticklabels(labels)
ax.set_ylabel('Prezzo'); ax.set_title('Confronto Heston vs Merton')
ax.legend(); ax.grid(alpha=0.3, axis='y')
plt.tight_layout(); plt.show()

## Commenti

### Calibrazione

- **Heston** (5 parametri) fitta meglio lo smile di mercato perché ha più gradi di libertà. La correlazione negativa ($\rho < 0$) genera lo skew.
- **Merton** (4 parametri) fatica sulle code e sullo skew perché ha un'unica componente di salto. Il fit è peggiore specialmente per strike bassi (put OTM).

### Coerenza MC vs analitico

- I prezzi MC delle vanilla sono coerenti con quelli COS (differenze dell'ordine dell'errore MC), confermando che le simulazioni sono corrette.

### Opzioni esotiche

1. **Asian call** < Vanilla call per entrambi i modelli: la media riduce la volatilità effettiva.

2. **Down-and-Out call** < Vanilla call: la barriera elimina i path che scendono sotto B=80, riducendo il valore.

3. Le **differenze tra modelli** sono più marcate per le esotiche che per le vanilla, perché:
   - Heston ha volatilità stocastica continua → path più "lisci"
   - Merton ha salti discreti → path con discontinuità che possono superare la barriera istantaneamente
   - Questo impatta soprattutto la D&O call, dove i salti del Merton possono far toccare la barriera più facilmente

4. Questo conferma che **modelli calibrati sulle stesse vanilla possono produrre prezzi diversi sulle esotiche**, ed è il punto chiave dell'esercizio.